In [1]:
# !pip install bayesian-optimization

In [2]:
import deepquantum as dq
import deepquantum.photonic.circuit as circuit
import matplotlib.pyplot as plt
import numpy as np

from deepquantum.optimizer import *
from deepquantum.photonic.decompose import *
from scipy.stats import unitary_group

np.set_printoptions(precision=8, floatmode='fixed', suppress=True) # to make the print info aligned

N = 8
u8x8 = unitary_group.rvs(N)
decomp_rlt = UnitaryDecomposer(u8x8, "cssr").decomp()
mzi_info = decomp_rlt[0]

def zero_init(mzi_info):
    for i in range(len(mzi_info['MZI_list'])):
        mzi_info['MZI_list'][i][2] = 0.0
        mzi_info['MZI_list'][i][3] = 0.0
    mzi_info['phase_angle_ori'] = np.zeros(N)
    mzi_info['phase_angle'] = np.zeros(N)

def bar_securing(mzi_info):
    adjustable_ids = []
    for i in range(len(mzi_info['MZI_list'])):
        if i not in adjustable_ids:
            mzi_info['MZI_list'][i][-1] = np.pi
            mzi_info['MZI_list'][i][-2] = np.pi
    
def xx_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi/2

def yy_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi/2
        mzi_info['MZI_list'][i][-2] = np.pi

def zz_planting(mzi_info):
    for i in [20, 24]:
        mzi_info['MZI_list'][i][-1] = np.pi
        mzi_info['MZI_list'][i][-2] = np.pi

def zx_planting(mzi_info):
    mzi_info['MZI_list'][20][-1] = np.pi
    mzi_info['MZI_list'][20][-2] = np.pi
    mzi_info['MZI_list'][24][-1] = np.pi/2


def trial_planting(mzi_info,angle_list):
    change_list = [[1,-1],
                   [4,-1],
                   [5,-1]]
    for i in range(len(angle_list)):
        a,b = change_list[i]
        mzi_info['MZI_list'][a][b] = angle_list[i]
        
zero_init(mzi_info)
bar_securing(mzi_info)
zz_planting(mzi_info)


def post_selection(rlt_strkey):
    # print(rlt_strkey)
    label_list = ['|00101000>', '|00100100>', '|00011000>', '|00010100>']
    value = np.zeros(4)
    for i in range(4):
        try:
            value[i] = rlt_strkey[label_list[i]]
        except:
            pass
    return value


def estimate_energy(post_selection_rlt,type='ZZ'):
    norm = post_selection_rlt/(post_selection_rlt.sum()+1e-16)
    if type=='ZZ': # ZZ basis
        return norm[0]+norm[3]-norm[1]-norm[2]
    if type=='ZI': # ZI basis
        return norm[0]+norm[1]-norm[2]-norm[3]
    if type=='IZ': # IZ basis
        return -norm[0]-norm[1]+norm[2]+norm[3]

def compute_process(mzi_info):
    cir = circuit.QumodeCircuit(nmode=8,init_state=[0,0, 0, 1, 0, 1, 0,0], name="test", cutoff = 3, basis = True) # basis=True, using state list 
    for info in mzi_info['MZI_list']:
        cir.ps(inputs=info[2],wires=[info[0]])
        cir.bs_theta(inputs=np.pi/4,wires=[info[0],info[1]])
        cir.ps(inputs=info[3],wires=[info[0]])
        cir.bs_theta(inputs=np.pi/4,wires=[info[0],info[1]])
    rlt = cir()
    rlt_strkey = dict()
    for key in rlt.keys():
        rlt_strkey[str(key)] = np.abs(rlt[key].detach().numpy().squeeze())**2

    # rltm = cir.measure(shots=10000)[0]
    # rltm_strkey = dict()
    # for key in rltm.keys():
    #     rltm_strkey[str(key)] = rltm[key]
    # return post_selection(rltm_strkey)

    return post_selection(rlt_strkey)

def estimate_eigval(angle_list):
    # to minimize
    zero_init(mzi_info)
    bar_securing(mzi_info)
    trial_planting(mzi_info,angle_list)

    zz_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_zz = estimate_energy(post_selection_rlt)

    xx_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_xx = estimate_energy(post_selection_rlt)

    yy_planting(mzi_info)
    post_selection_rlt = compute_process(mzi_info)
    e_yy = estimate_energy(post_selection_rlt)
    
    return (1 + e_zz - e_xx - e_yy) / 2  

    # zz_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_zi = estimate_energy(post_selection_rlt,type='ZI')

    # xx_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_ix = estimate_energy(post_selection_rlt,type='IZ')

    # zx_planting(mzi_info)
    # post_selection_rlt = compute_process(mzi_info)
    # e_zx = estimate_energy(post_selection_rlt,type='ZZ')
    
    # return (1 + e_zi + e_ix - e_zx) / 2  


def target_function(angle_list):
    # to maximize
    return -estimate_eigval(angle_list)

def void_target():
    return -1

In [6]:
# 测试BO算法
print("="*50)
print("运行BO")
param_init = {'t1':0, 't2':0, 't3':0}
bo_optimizer = OptimizerBayesian(target_func=void_target,param_init=param_init,random_state=0)

print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(100):
    # 待片上计算的参数
    p1 = bo_optimizer.param_suggest()
    # 模拟芯片计算过程，之后需要替换成接口
    f1 = -estimate_eigval(p1.tolist()) #  BO 内置用法是最大化目标；但是接下来打印时再添一个符号即可
    # 测试结果传回，更新param_dict
    bo_optimizer.param_register([p1],[f1])
    f1_to_print = -f1
    if bo_optimizer.best_target == f1:
        mark = "*"
    else:
        mark = " "
    if f1_to_print<0:
        print(bo_optimizer.iter,"\t",p1,"\t",f"{f1_to_print:8.10f} ",mark)
    else:
        print(bo_optimizer.iter,"\t",p1,"\t ",f"{f1_to_print:8.10f} ",mark)

print("BO结果：",estimate_eigval(list(bo_optimizer.best_param_dict.values())))

运行BO
轮次 	 参数值 	                                 目标值


1 	 [3.44829694 4.49366732 3.78727399] 	  0.3947972653  *
2 	 [0.04130112 0.38026479 0.23805479] 	  0.1257407466  *
3 	 [1.44694751 6.01625953 2.40802370] 	  0.9167352244   
4 	 [3.39985388 3.47527230 4.60283868] 	  0.9266207489   
5 	 [1.22734700 1.73043008 0.54248881] 	 -0.6500056711  *
6 	 [2.09069562 1.41932526 0.00000000] 	 -0.7981043946  *
7 	 [2.88304816 2.97512045 0.00000000] 	 -0.9091092168  *
8 	 [5.18323663 1.69216123 0.00000000] 	  0.8385735512   
9 	 [1.38723141 3.44086540 0.00000000] 	  0.2468208299   
10 	 [2.63688277 2.14576832 0.98241740] 	  0.2032860750   
11 	 [3.36385140 3.65330729 0.00000000] 	 -0.7218107201   
12 	 [1.78490675 0.45700257 0.35228934] 	 -0.3414096053   
13 	 [4.20399059 5.04937878 0.00000000] 	 -0.7101006858   
14 	 [5.48218048 5.61032775 0.00000000] 	 -0.9820068680  *
15 	 [5.03704579 6.28318531 0.99640996] 	  0.5997025340   
16 	 [5.36604905 4.69659434 0.00000000] 	 -0.7862136042   
17 	 [6.23089379 5.28085801 0.56497262] 	  0.1455149062   
18 	 [

In [4]:
# 测试SPSA算法
print("="*50)
print("运行SPSA")
param_init = {'t1':np.random.randn(), 't2':np.random.randn(), 't3':np.random.randn()}
spsa_optimizer = OptimizerSPSA(target_func=void_target,param_init=param_init,random_state=0)

print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(200):
    # 待片上计算的两组参数
    p1,p2 = spsa_optimizer.param_suggest()
    # 以下两行模拟芯片计算过程，之后需要替换成接口
    f1 = estimate_eigval(p1.tolist())
    f2 = estimate_eigval(p2.tolist())
    # 测试结果传回，更新param_dict
    spsa_optimizer.param_register([p1,p2],[f1,f2])
    # spsa_optimizer.param_register(np.array([p1,p2]),np.array([f1,f2]))
    

    if f1<0:
        print(spsa_optimizer.iter,"\t",p1,"\t",f"{f1:8.10f} ")
    else:
        print(spsa_optimizer.iter,"\t",p1,"\t ",f"{f1:8.10f} ")
    if f2<0:
        print(spsa_optimizer.iter,"\t",p2,"\t",f"{f2:8.10f} ")
    else:
        print(spsa_optimizer.iter,"\t",p2,"\t ",f"{f2:8.10f} ")
    # print(spsa_optimizer.hyperparam['c']/(1+spsa_optimizer.iter)**spsa_optimizer.hyperparam['gamma'])

    if (f1<-0.92) and (f2<-0.92):
        # print("!!!进入步长减小阶段!!!")
        spsa_optimizer.hyperparam['c'] = 0.001
    elif (f1<-0.999) and (f2<-0.999):
        spsa_optimizer.hyperparam['c'] = 1e-4

print("SPSA结果：",estimate_eigval(list(spsa_optimizer.best_param_dict.values())))

运行SPSA
轮次 	 参数值 	                                 目标值
1 	 [ 0.12604016  0.56353842 -0.19234807] 	 -0.2716579327 
1 	 [ 0.10604016  0.58353842 -0.17234807] 	 -0.2368486010 
2 	 [ 0.13251132  0.55706726 -0.19881923] 	 -0.2815470800 
2 	 [ 0.11386359  0.57571499 -0.18017150] 	 -0.2511828906 
3 	 [ 0.12090446  0.55077463 -0.20511187] 	 -0.2414854959 
3 	 [ 0.13880395  0.56867412 -0.18721237] 	 -0.3120761031 
4 	 [ 0.13725956  0.56712973 -0.17136987] 	 -0.3290447820 
4 	 [ 0.15464645  0.58451662 -0.18875676] 	 -0.3470358360 
5 	 [ 0.15866422  0.57153497 -0.17577511] 	 -0.3787760432 
5 	 [ 0.14166481  0.58853439 -0.19277453] 	 -0.3072688953 
6 	 [ 0.17557938  0.57130907 -0.15885996] 	 -0.4438339907 
6 	 [ 0.15889013  0.55461982 -0.17554921] 	 -0.3858477528 
7 	 [ 0.18950900  0.56880727 -0.14493033] 	 -0.4967707240 
7 	 [ 0.17307758  0.58523869 -0.16136175] 	 -0.4279756788 
8 	 [ 0.19007913  0.55202584 -0.12814891] 	 -0.5279164826 
8 	 [ 0.20629043  0.56823714 -0.14436021] 	 -0.5378905594 
9 

In [5]:
# 测试Fourier级数方法
print("="*50)
print("运行Fourier级数法")
param_init = {'t1':np.random.randn(), 't2':np.random.randn(), 't3':np.random.randn()}
# param_init = dict(zip(param_init.keys(),np.random.randn(3)))
fourier_optimizer = OptimizerFourier(target_func=void_target,param_init=param_init,random_state=0,order=3,lr=0.1)
print("轮次","\t","参数值","\t"," "*31,"目标值")
for _ in range(50):
    # 待片上计算的两组参数
    param_array = fourier_optimizer.param_suggest()
    # 模拟芯片计算过程，之后需要替换成接口
    target = np.zeros(len(param_array))
    for i in range(len(param_array)):
        target[i] = estimate_eigval(param_array[i])

    # 测试结果传回，更新param_dict
    fourier_optimizer.param_register(param_array,target)
    p1 = np.array(list(fourier_optimizer.best_param_dict.values()))
    f1 = fourier_optimizer.best_target
    if f1<0:
        print(fourier_optimizer.iter,"\t",p1,"\t",f"{f1:8.10f} ")
    else:
        print(fourier_optimizer.iter,"\t",p1,"\t ",f"{f1:8.10f} ")

print("Fourier结果：",estimate_eigval(list(fourier_optimizer.best_param_dict.values())))

运行Fourier级数法
轮次 	 参数值 	                                 目标值
1 	 [0.15650654 0.23218104 0.00000000] 	 -0.9261000639 
2 	 [0.24215053 0.31843215 0.00000000] 	 -0.9627085499 
3 	 [0.36816295 0.44202150 0.00000000] 	 -0.9825996643 
4 	 [0.48253207 0.53621082 0.00000000] 	 -0.9939607867 
5 	 [0.58076884 0.61726507 0.00000000] 	 -0.9979074916 
6 	 [0.66189373 0.68684693 0.00000000] 	 -0.9992018447 
7 	 [0.72702523 0.74424278 0.00000000] 	 -0.9996709109 
8 	 [0.77912714 0.79110559 0.00000000] 	 -0.9998564422 
9 	 [0.82125658 0.82965201 0.00000000] 	 -0.9999347453 
10 	 [0.85587451 0.86179790 0.00000000] 	 -0.9999693953 
11 	 [0.88480355 0.88900769 0.00000000] 	 -0.9999852905 
12 	 [0.90936326 0.91236305 0.00000000] 	 -0.9999927905 
13 	 [0.93051067 0.93266128 0.00000000] 	 -0.9999964111 
14 	 [0.94894796 0.95049632 0.00000000] 	 -0.9999981874 
15 	 [0.96519833 0.96631738 0.00000000] 	 -0.9999990743 
16 	 [0.97965793 0.98046950 0.00000000] 	 -0.9999995225 
17 	 [0.99263158 0.99322196 0.0000000